In [1]:
import time
s = time.time()

In [2]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [4]:
df = pd.merge(listings, reviews, on='listing_id')

In [5]:
df.describe();

In [6]:
df.head();

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['comments']))

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['lisa', 'is', 'wonderful', 'kind', 'and', 'thoughtful', 'host', 'the', 'listing', 'is', 'accurate', 'there', 'is', 'an', 'entire', 'floor', 'for', 'the', 'guest', 'with', 'full', 'bath', 'and', 'kitchen', 'area', 'looking', 'out', 'over', 'leafy', 'backyard', 'everything', 'needed', 'for', 'my', 'one', 'month', 'stay']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# print('type(data_words_nostops): ', type(data_words_nostops))

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# print('type(data_words_bigrams): ', type(data_words_bigrams))
# print(data_words_bigrams)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

In [12]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [13]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['wonderful', 'kind', 'thoughtful', 'host', 'list', 'accurate', 'entire', 'floor', 'guest', 'full', 'area', 'look', 'backyard', 'need', 'month', 'stay']]


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]]


In [15]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=2,
                                           per_word_topics=True)

In [16]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.574*"stay" + 0.112*"amazing" + 0.085*"love" + 0.063*"city" + 0.056*"want" '
  '+ 0.030*"absolutely" + 0.029*"ever" + 0.017*"explore" + '
  '0.010*"recommendation" + 0.005*"favorite"'),
 (1,
  '0.475*"host" + 0.127*"day" + 0.067*"arrival" + 0.056*"cancel" + '
  '0.055*"reservation" + 0.033*"quite" + 0.025*"long" + 0.024*"appreciate" + '
  '0.019*"pleasant" + 0.019*"flat"'),
 (2,
  '0.270*"nice" + 0.261*"room" + 0.194*"really" + 0.064*"people" + '
  '0.060*"small" + 0.060*"price" + 0.044*"big" + 0.036*"house" + '
  '0.003*"traveller" + 0.000*"say"'),
 (3,
  '0.473*"apartment" + 0.109*"locate" + 0.105*"spacious" + 0.103*"wonderful" + '
  '0.052*"exactly" + 0.034*"describe" + 0.025*"option" + 0.024*"return" + '
  '0.021*"perfectly" + 0.018*"transportation"'),
 (4,
  '0.191*"make" + 0.151*"feel" + 0.137*"home" + 0.085*"safe" + 0.085*"right" + '
  '0.069*"lovely" + 0.064*"sure" + 0.064*"stay" + 0.060*"welcome" + '
  '0.039*"touch"'),
 (5,
  '0.228*"need" + 0.206*"time" + 0.062*"thi

In [17]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.055515820099835

Coherence Score:  0.5800706477447359


In [18]:
print((s-time.time())/60/60)

-7.107343808081414


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/tristandealwis/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
18     0.053027  0.039253       1        1  5.979722
14     0.052643 -0.013130       2        1  5.427166
12     0.052242  0.038209       3        1  5.330859
13     0.051886  0.052561       4        1  5.191372
7      0.051972  0.050048       5        1  5.187817
0     -0.346062 -0.001787       6        1  5.112792
17     0.052222  0.039175       7        1  5.044379
9      0.052891 -0.194440       8        1  5.019247
4     -0.304349 -0.000492       9        1  4.984803
5     -0.236766  0.000919      10        1  4.980690
2      0.052412  0.023950      11        1  4.978749
16     0.052187  0.041145      12        1  4.881724
8      0.052240  0.038062      13        1  4.840817
6      0.052957 -0.411364      14        1  4.837861
19     0.052509  0.012400      15        1  4.826185
15     0.051812  0.054214      16        1  4.806359
3      0.051301  0.060988      17        1  4.766871
11     0.051956  0.067433      18        1  4.704207
1      0.052220  0.039359      19        1  4.586359
10     0.050700  0.063497      20        1  4.512013, topic_info=    Category           Freq       Term          Total  loglift  logprob
34   Default  685427.000000      place  685427.000000  30.0000  30.0000
13   Default  767704.000000       stay  767704.000000  29.0000  29.0000
60   Default  593169.000000      great  593169.000000  28.0000  28.0000
22   Default  508803.000000      clean  508803.000000  27.0000  27.0000
20   Default  503121.000000  apartment  503121.000000  26.0000  26.0000
..       ...            ...        ...            ...      ...      ...
450  Topic20    4348.236328       taxi    4349.714355   3.0981  -5.4452
175  Topic20    3750.931152  agradable    3752.409180   3.0980  -5.5930
497  Topic20    3736.546387    present    3738.024414   3.0980  -5.5968
390  Topic20    3582.580078       como    3584.058105   3.0980  -5.6389
478  Topic20    3389.033936        pie    3390.511963   3.0980  -5.6945

[884 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
123       5  0.999948                able
479       6  0.999966          absolutely
16       12  0.999977              access
246      20  0.999939          accessible
17       15  0.999972         accommodate
...     ...       ...                 ...
136       3  0.999879                wish
281      18  0.998994  without_hesitation
15       17  0.999987           wonderful
368       1  0.999972                work
73       15  0.999995               would

[430 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 15, 13, 14, 8, 1, 18, 10, 5, 6, 3, 17, 9, 7, 20, 16, 4, 12, 2, 11])